In [1]:
import os as os
import requests as req
import zipfile as zipfile
from datetime import datetime

In [2]:
def data_directories(**kwargs):
    """
    Funcao de criacao das pastas para armazenamento dos dados, caso nao existam.
    
    Args:
        directories (list): Lista do caminho dos diretorios a serem criados.
        
    Return:
        Cria os diretorios para armazenamento dos dados.  
    """

    # Diretorios para armazenamento dos dados coletados
    kwargs['directories'] = ['..\\data\\01-collected'
                            ,'..\\data\\02-cleaned'
                            ,'..\\data\\03-organized'
                            ,'..\\data\\04-standardized']
    
    # Log: Mensagem de inicio da criacao dos diretorios
    print(str(datetime.now()) + ': Inicio da criacao dos diretorios para armazenamento dos dados.')
    
    # Para cada diretorio na lista de diretorios
    for directory in kwargs['directories']:
        
        # Verifica se o diretorio existe
        if not os.path.exists(directory):
            
            # Cria o diretorio
            os.makedirs(directory)
            
            # Log: Mensagem de criacao do diretorio
            print(str(datetime.now()) + ': Diretorio ' + directory + ' criado.')
        
        else:
            
            # Log: Mensagem de diretorio existente
            print(str(datetime.now()) + ': Diretorio ' + directory + ' ja existe.')
    
    # Log: Mensagem de fim da criacao dos diretorios
    print(str(datetime.now()) + ': Fim da criacao dos diretorios para armazenamento dos dados.')

In [3]:
def download_file(**kwargs):
    """
    Funcao que realiza o download do arquivo, extrai os arquivos compactados e organiza-os em
    um unico diretorio. Ao final do download, e verificado 
    se o arquivo solicitado esta vazio. Caso esteja vazio, o arquivo e apagado.
    
    Args:
        url_query (str): URL parametrizada para download do arquivo.
        download_path (str): Caminho do diretorio onde sera salvo o arquivo.
        download_directory (str): Caminho do diretorio para decompactacao do arquivo
    """
    
    # Log: Mensagem de inicio do download do arquivo
    print(str(datetime.now()) + ': Inicio do download do arquivo ' + kwargs['download_path'])
    
    # Prepara a requisicao de download do arquivo
    res = req.get(kwargs['url_query'], stream = True)
    
    # Se for encontrada a URL para download
    if res.status_code == 200:
        
        # Log: Mensagem de URL encontrada e de inicio do download
        print(str(datetime.now()) + ': URL encontrada, fazendo o download do arquivo: ' + kwargs['download_path'])
        
        # Faz o download do arquivo
        with open(kwargs['download_path'], 'wb') as f:
            
            # Download do arquivo em pedacos
            for chunk in res.iter_content(chunk_size = 1024):
                f.write(chunk)
        
        # Verifica o tamanho do arquivo recebido
        file_size = os.stat(kwargs['download_path']).st_size
        
        # Se o tamanho do arquivo for muito 
        # pequeno remove o arquivo da pasta 
        if file_size <= 100:
            
            # Log: Mensagem de remocao por arquivo por nao conter dados
            print(str(datetime.now()) + ': Arquivo ' + kwargs['download_path'] + ' removido por conter poucos dados.')
            
            # Remove o arquivo sem dados
            os.remove(kwargs['download_path'])
            
        # Caso contrario, descompacta o arquivo, organiza-os 
        # no diretorio e remove o arquivo compactado
        else:
            
            # Log: Mensagem de inicio da descompactacao do arquivo zip
            print(str(datetime.now()) + ': Descompactando o arquivo ' + kwargs['download_path'])
            
            # Descompacta o arquivo zip mantem apenas o csv
            zip_ref = zipfile.ZipFile(kwargs['download_path'], 'r')
            zip_ref.extractall(path=kwargs['download_directory'])
            zip_ref.close()
            os.remove(kwargs['download_path'])
            
            # Log: Mensagem de inicio da organizacao dos arquivos
            print(str(datetime.now()) + ': Organizando os arquivos descompactados de: ' + kwargs['download_path'])
            
            # log: Mensagem de finalizacao do download do arquivo
            print(str(datetime.now()) + ': Fim do download do arquivo ' + kwargs['download_path'])

In [4]:
def collect_data(**kwargs):
    """
    Coleta os dados de cada URL de referencia contida na lista informada.
    
    Args:
        url_list (list): Lista de URLs para download.
    """
    
    # Cria os diretorios para armazenamento dos dados
    data_directories(**kwargs)
    
    # Define o caminho completo para download dos arquivos
    kwargs['download_directory'] = '..\\data\\01-collected\\'
    
    # Define o nome do arquivo para download
    kwargs['file_name'] = '_CNPJ'
    kwargs['file_ext'] = '.zip'
       
    # Para cada URL de referencia dentro da lista
    for i, url_ref in enumerate(kwargs['url_list']):
        
        # Prepara o nome do arquivo para download
        kwargs['download_file'] = str(i) + kwargs['file_name'] + kwargs['file_ext']
        
        # Define a URL completa para download do arquivo
        kwargs['url_query'] = url_ref
        
        # Define o caminho completo para armazenamento do arquivo de download
        kwargs['download_path'] = kwargs['download_directory'] + kwargs['download_file']
        
        # Inicia o download do arquivo
        download_file(**kwargs)

In [6]:
def main():
    """
    Coleta os dados do Dados Públicos de CNPJ
    Origem: REceita Federal (receita.economia.gov.br)
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Lista de URLs para download dos arquivos zip
    kwargs['url_list'] = ['http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_01.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_02.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_03.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_04.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_05.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_06.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_07.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_08.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_09.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_10.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_11.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_12.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_13.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_14.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_15.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_16.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_17.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_18.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_19.zip'
                         ,'http://200.152.38.155/CNPJ/DADOS_ABERTOS_CNPJ_20.zip'
                         ]
                          
    # Coleta os arquivos
    collect_data(**kwargs)

if __name__ == '__main__':
    main()

2019-08-07 18:54:43.772392: Inicio da criacao dos diretorios para armazenamento dos dados.
2019-08-07 18:54:43.773391: Diretorio ..\data\01-collected criado.
2019-08-07 18:54:43.774388: Diretorio ..\data\02-cleaned criado.
2019-08-07 18:54:44.097553: Diretorio ..\data\03-organized criado.
2019-08-07 18:54:44.098520: Diretorio ..\data\04-standardized criado.
2019-08-07 18:54:44.098520: Fim da criacao dos diretorios para armazenamento dos dados.
2019-08-07 18:54:44.098520: Inicio do download do arquivo ..\data\01-collected\0_CNPJ.zip
2019-08-07 18:54:46.079909: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\0_CNPJ.zip
2019-08-07 21:04:43.443068: Descompactando o arquivo ..\data\01-collected\0_CNPJ.zip
2019-08-07 21:05:20.831825: Organizando os arquivos descompactados de: ..\data\01-collected\0_CNPJ.zip
2019-08-07 21:05:20.831825: Fim do download do arquivo ..\data\01-collected\0_CNPJ.zip


In [ ]:
data_directories(**kwargs)